# 플래너 알아보기

지금까지는 작업을 수행하기 위해 함수와 프롬프트를 수동으로 호출했습니다. 그러나 시맨틱 커널 SDK는 AI를 사용하여 지정된 시나리오에 적절한 플러그 인을 자동으로 호출하는 플래너를 지원합니다.

### 플래너란?
플래너란 사용자의 요청을 실행 가능한 계획으로 변환하는 동적 도구입니다. 플래너는 사용자나 개발자의 요청일 수 있는 목표를 파악합니다. 목표를 달성하기 위해 플래너는 AI를 이용해 커널에 등록된 플러그 인을 선택하고 일련의 단계로 결합합니다.

레시피와 쇼핑 목록을 추천하고 사용자가 주방에 가지고 있는 재료를 관리하는 플러그 인을 만들었다고 가정해 보세요. 플래너는 동적 사용자 입력을 위한 워크플로를 자동으로 만들 수 있습니다. 예를 들어, 사용자는 "치킨 티카 마살라를 만들려면 무엇이 필요하나요?"와 같은 요청을 입력할 수 있습니다. 플래너는 해당 시나리오에 대해 명시적으로 호출하지 않고도 올바른 함수를 자동으로 결합할 수 있습니다.

### 작동 방식

뒤에서 플래너는 LLM(대규모 언어 모델)에 대한 프롬프트를 사용하여 계획을 생성합니다. 플래너에게는 계획을 생성할 때 사용할 일련의 규칙이 제공됩니다. 규칙에는 계획을 만들기 위해 커널에 제공한 모든 함수에 대한 설명이 포함됩니다. 이 함수는 네이티브 함수에 설명을 포함하는 이유 중 일부입니다. 이러한 설명은 LLM이 계획을 생성할 때 사용할 함수를 결정하는 데 도움이 됩니다.

다음은 Handlebars 플래너가 계획을 생성하는 데 사용하는 LLM 프롬프트의 일부입니다.

```text
## Start
Now take a deep breath and accomplish the task:
1. Keep the template short and sweet. Be as efficient as possible.
2. Do not make up helpers or functions that were not provided to you, and be especially careful to NOT assume or use any helpers or operations that were not explicitly defined already.
3. If you can't fully accomplish the goal with the available helpers, just print "{{insufficientFunctionsErrorMessage}}".
4. Always start by identifying any important values in the goal. Then, use the `\{{set}}` helper to create variables for each of these values.
5. The template should use the \{{json}} helper at least once to output the result of the final step.
6. Don't forget to use the tips and tricks otherwise the template will not work.
7. Don't close the ``` handlebars block until you're done with all the steps.
```

플래너를 사용하면 복잡한 시나리오를 처리할 수 있는 고급 애플리케이션을 쉽게 만들 수 있습니다.

## 플래너 사용 방법

플래너는 함수와 프롬프트를 자동으로 실행하는 데 유용한 도구입니다. 현재 의미 체계 커널 플래너를 미리 볼 수 있습니다. 핸들바 플래너를 사용하려면 먼저 추가 기능 패키지를 설치해야 합니다.

그런 다음 `CreatePlanAsync`를 사용하여 새 플래너 개체를 만들고 여기에 목표를 전달합니다. 목표는 원하는 결과를 설명하는 문자열입니다. 그러면 플래너가 목표를 달성하기 위한 계획을 만듭니다. `CreatePlanAsync`는 사용자가 계획을 호출하고 실행하는 데 사용할 수 있는 `HandlebarsPlan` 개체를 반환합니다.

예를 들어 다음 함수를 지원하는 몇 가지 성분 플러그 인이 있다고 가정해 보겠습니다.

- `GetIngredients`: 사용자가 가지고 있는 재료 목록을 가져옵니다
- `GetRecipe`: 주어진 레시피에 필요한 재료 목록을 가져옵니다
- `GetMissingIngredients`: 주어진 레시피에 필요한 재료 중 사용자의 주방에 없는 재료 목록을 가져옵니다


```csharp
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

string goal = @"What ingredients is the user missing from their 
   current ingredients list to make a recipe for blueberry muffins";

var plan = await planner.CreatePlanAsync(kernel, goal);
var result = await plan.InvokeAsync(kernel);
Console.WriteLine(result);
```

결과는 다음 출력과 유사합니다.

> Based on the list of ingredients for blueberry muffins, you are missing blueberries and salt from your available ingredients.

생성된 계획을 저장하고 나중에 다시 로드할 수도 있습니다. 예를 들면 다음과 같습니다.

```csharp
var plan = await planner.CreatePlanAsync(kernel, goal);
var serializedPlan = plan.ToString();

HandlebarsPlan reloadedPlan = new HandlebarsPlan(serializedPlan);

var result = await reloadedPlan.InvokeAsync(kernel);
Console.WriteLine($"Results: {result}");
```

핸들바 플래너는 개발자가 동적 애플리케이션을 만들고 코드가 적은 복잡한 시나리오를 처리하는 데 도움이 되는 강력한 기능입니다. 


### 콘서트 제안 계획 작성

1. Handlebars 플래너를 설치합니다. 

In [15]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview"
#r "nuget: Microsoft.SemanticKernel, 1.2.0"

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.ComponentModel;
using System.Text.Json;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var builder = Kernel.CreateBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.2.0-preview

2. `Plugins` 폴더에 `MusicConcertPlugin.cs`라는 새 파일을 만들고 코드를 추가했습니다.

```csharp
using System.ComponentModel;
using Microsoft.SemanticKernel;

public class MusicConcertPlugin
{
    [KernelFunction, Description("Get a list of upcoming concerts")]
    public static string GetTours()
    {
        string dir = Directory.GetCurrentDirectory();
        string content = File.ReadAllText($"{dir}/data/concertdates.txt");
        return content;
    }
}
```

`GetTours` 함수는 'concertdates.txt'라는 파일을 읽고 콘텐츠를 반환합니다. 이 함수는 예정된 콘서트 목록을 검색하는 데 사용됩니다.

다음으로, 사용자가 최근에 연주한 음악을 기반으로 콘서트를 제안하도록 LLM에 요청하는 프롬프트를 만듭니다.

3. 'Prompts' 폴더에 'SuggestConcert'라는 새 폴더를 만듭니다.
4. 다음 콘텐츠로 'SuggestConcert' 폴더에 'config.json' 파일을 만듭니다.

```json
{
    "schema": 1,
    "type": "completion",
    "description": "Suggest a concert to the user",
    "execution_settings": {
        "default": {
            "max_tokens": 4000,
            "temperature": 0
        }
    },
    "input_variables": [
        {
            "name": "upcomingConcerts",
            "description": "A list of artist's upcoming concerts",
            "required": true
        },
        {
            "name": "recentlyPlayedSongs",
            "description": "A list of songs recently played by the user",
            "required": true
        },
        {
            "name": "location",
            "description": "The user's location",
            "required": true
        }
    ]
}
```
5. 다음 콘텐츠로 `SuggestConcert` 폴더에 `skprompt.txt` 파일을 만듭니다.

```text
Based on the user's recently played songs:
{{$recentlyPlayedSongs}}

And a list of upcoming concerts:
{{$upcomingConcerts}}

Suggest an upcoming concert. The user lives in {{$location}}, 
please recommend a relevant concert that is close to their location.
```

이 프롬프트는 사용자가 최근에 재생한 노래와 위치를 기반으로 콘서트를 제안하도록 LLM에 요청합니다. 다음으로 계획 도구를 호출하여 목표를 달성하기 위해 플러그 인을 결합하는 계획을 만듭니다.


In [16]:
#!import Plugins/MusicConcertPlugin.cs
#!import Plugins/MusicLibraryPlugin.cs

kernel.ImportPluginFromType<MusicLibraryPlugin>();
kernel.ImportPluginFromType<MusicConcertPlugin>();
kernel.ImportPluginFromPromptDirectory("Prompts");

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

string location = "Redmond WA USA";
string goal = @$"사용자가 최근에 재생한 음악을 바탕으로, ${location}에 거주하는 사용자를 위한 콘서트를 추천해 주세요. ";

var plan = await planner.CreatePlanAsync(kernel, goal);
var result = await plan.InvokeAsync(kernel);


Console.WriteLine($"Results: {result}");

Results: Based on your location in Redmond, WA, and your recently played songs, I recommend attending the Clairo concert in Seattle, WA. Here are the details:

- **Artist:** Clairo
- **Location:** Seattle, WA, USA
- **Date:** February 14, 2024

Clairo's music aligns with your interest in alternative and indie genres, making this concert a great fit for you. Enjoy the show!


Handlebars 플래너에서 생성된 계획을 조정할 수 있습니다. 이 연습에서는 계획을 생성하고, 템플릿을 조정하고, 이를 함수 프롬프트로 사용합니다.

In [17]:
Console.WriteLine($"Concert Plan: {plan}");

Concert Plan: {{!-- Step 1: Specify user's location --}}
{{set "location" "Redmond WA USA"}}

{{!-- Step 2: Retrieve recently played songs --}}
{{set "recentlyPlayed" (MusicLibraryPlugin-GetRecentPlays)}}

{{!-- Step 3: Retrieve upcoming concerts --}}
{{set "upcomingConcerts" (MusicConcertPlugin-GetTours)}}

{{!-- Step 4: Suggest a concert based on recently played songs and user's location --}}
{{set "concertSuggestion" (Prompts-SuggestConcert upcomingConcerts=upcomingConcerts recentlyPlayedSongs=recentlyPlayed location=location)}}

{{!-- Step 5: Output the suggested concert --}}
{{json concertSuggestion}}


다음 코드를 추가합니다. 이 코드에서는 사용자에게 노래를 제안하는 프롬프트에서 함수를 만듭니다. 그런 다음 커널 플러그 인에 추가합니다. 마지막으로 사용자에게 노래를 제안한다는 목표를 달성하기 위한 계획을 세우도록 플래너에게 지시합니다.

In [18]:
#pragma warning disable SKEXP0060

var songSuggesterFunction = kernel.CreateFunctionFromPrompt(
    promptTemplate: @"Based on the user's recently played music:
        {{$recentlyPlayedSongs}}
        recommend a song to the user from the music library:
        {{$musicLibrary}}",
    functionName: "SuggestSong",
    description: "Suggest a song to the user"
);

kernel.Plugins.AddFromFunctions("SuggestSongPlugin", [songSuggesterFunction]);

var songSuggestPlan = await planner.CreatePlanAsync(kernel, @"사용자가 최근에 재생한 노래를 바탕으로 음악 라이브러리에서 노래를 추천해 주세요.");

Console.WriteLine("Song Plan:");
Console.WriteLine(songSuggestPlan);

Song Plan:
{{!-- Step 1: Get the list of recently played songs --}}
{{set "recentlyPlayedSongs" (MusicLibraryPlugin-GetRecentPlays)}}

{{!-- Step 2: Get the user's music library --}}
{{set "musicLibrary" (MusicLibraryPlugin-GetMusicLibrary)}}

{{!-- Step 3: Suggest a song based on recently played songs and the music library --}}
{{set "recommendedSong" (SuggestSongPlugin-SuggestSong recentlyPlayedSongs=recentlyPlayedSongs musicLibrary=musicLibrary)}}

{{!-- Step 4: Output the recommended song --}}
{{json recommendedSong}}


다음으로 만들어진 템플릿을 사용하여 고유한 Handlebars 계획을 만듭니다. 사용자가 콘서트를 요청하는 경우 콘서트 계획의 단계를 사용하고, 사용자가 노래를 요청하는 경우 노래 계획의 단계를 사용하려고 합니다.

4. 다음 텍스트를 사용하여 `handlebarsTemplate.txt`라는 새 파일을 만듭니다. 

    {{#if ...}} 구문을 확인합니다. 이 구문은 C#의 기존 if-else 블록과 유사하게 Handlebars 플래너가 사용할 수 있는 조건문 역할을 합니다. if 문은 {{/if}}로 닫혀야 합니다. 또한 사용자 위치를 location 변수에 설정하는 {{set ...}} 문을 추가합니다.

    이 템플릿에서 suggestConcert 변수가 true인 경우 플래너는 콘서트를 제안하는 단계를 완료합니다. 그렇지 않으면 노래 추천 단계가 완료됩니다. 

```
{{#if suggestConcert}}
    {{!-- Step 1: Identify key values --}}
    {{set "location" location}} 

    {{!-- Step 2: Call the 'MusicLibraryPlugin-GetRecentPlays' helper to get the recently played music --}}     
    {{set "recentlyPlayedSongs" (MusicLibraryPlugin-GetRecentPlays)}}

    {{!-- Step 3: Call the 'MusicConcertsPlugin-GetConcerts' helper to get the list of upcoming concerts --}}   
    {{set "upcomingConcerts" (MusicConcertsPlugin-GetConcerts)}}

    {{!-- Step 4: Call the 'Prompts-SuggestConcert' helper to suggest a concert based on the provided inputs --}}
    {{set "suggestedConcert" (Prompts-SuggestConcert input=location recentlyPlayedSongs=recentlyPlayedSongs upcomingConcerts=upcomingConcerts)}}

    {{!-- Step 5: Output the suggested concert --}}
    {{json suggestedConcert}}
{{else}}
    {{!-- Step 1: Identify key values --}}
    {{set "recentlyPlayedSongs" (MusicLibraryPlugin-GetRecentPlays)}}
    {{set "musicLibrary" (MusicLibraryPlugin-GetMusicLibrary)}}

    {{!-- Step 2: Call custom helper to suggest a song --}}
    {{set "suggestedSong" (SuggestSongPlugin-SuggestSong recentlyPlayedSongs musicLibrary)}}

    {{!-- Step 3: Output the suggested song --}}
    {{json suggestedSong}}
{{/if}}
```

이 템플릿에서 `suggestConcert` 변수가 true인 경우 계획 도구는 콘서트를 제안하는 단계를 완료합니다. 그렇지 않으면 노래 추천 단계가 완료됩니다.

5. 기존 코드를 수정하여 Handlebars 계획을 제거합니다.

In [19]:
string dir = Directory.GetCurrentDirectory();
kernel = builder.Build();

kernel.ImportPluginFromType<MusicLibraryPlugin>();
kernel.ImportPluginFromType<MusicConcertPlugin>();
kernel.ImportPluginFromPromptDirectory($"{dir}/Prompts");

var songSuggesterFunction = kernel.CreateFunctionFromPrompt(
    promptTemplate: @"Based on the user's recently played music:
    {{$recentlyPlayedSongs}}
    recommend a song to the user from the music library:
    {{$musicLibrary}}",
    functionName: "SuggestSong",
    description: "Suggest a song to the user"
);

kernel.Plugins.AddFromFunctions("SuggestSongPlugin", [songSuggesterFunction]);

템플릿 파일을 읽고 다음 코드를 사용하여 함수를 만듭니다. 이 코드에서는 `Template` 개체를 `TemplateFormat`과 함께 커널 메서드 `CreateFunctionFromPrompt`에 전달합니다. `CreateFunctionFromPrompt`는 또한 지정된 템플릿을 구문 분석하는 방법을 커널에 알려 주는 `IPromptTemplateFactory` 형식을 허용합니다. `Handlebars` 템플릿을 사용하고 있으므로 `HandlebarsPromptTemplateFactory` 형식을 사용합니다.

In [20]:
string dir = Directory.GetCurrentDirectory();
string template = File.ReadAllText($"{dir}/handlebarsTemplate.txt");

var handlebarsPromptFunction = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig() {
        Template = template,
        TemplateFormat = "handlebars"
    }, new HandlebarsPromptTemplateFactory()
);

다음으로 몇 가지 인수를 사용하여 함수를 실행하고 결과를 확인해 보겠습니다.

7. 다음 코드를 추가합니다. 생성된 출력은 최근에 재생된 음악을 기반으로 사용자에게 노래를 추천해야 합니다. 프롬프트는 최근 재생된 음악 목록을 기반으로 사용자에게 노래를 제안할 수 있었습니다. 또한 suggestConcert 변수를 true로 설정하여 어떤 일이 일어나는지 확인할 수도 있습니다!

In [21]:
string location = "Redmond WA USA";
var templateResult = await kernel.InvokeAsync(handlebarsPromptFunction,
    new () {
        { "location", location },
        { "suggestConcert", false }
    });

Console.WriteLine(templateResult);

Mitski's "Strawberry Blond" is a fantastic recommendation! Mitski is known for her evocative lyrics and distinctive sound that blends elements of indie, alternative, and folk music. If you appreciate the emotional depth and storytelling in "Loanh Quanh" by Mademoiselle, you're likely to enjoy Mitski's work as well. Dive in and enjoy the unique soundscape she creates!



## 플래너 최적화

플래너는 유용하게 활용할 수 있는 도구로, 미리 정의된 함수를 손쉽게 조합하고 복잡한 사용자 시나리오를 다양하게 지원할 수 있습니다. 그러나 플래너를 사용하기 전에 몇 가지 중요한 사항을 고려해야 합니다. 성능, 비용, 정확성에서 발생할 수 있는 잠재적인 단점을 이해하고 이를 관리하면 플래너를 최대한 효율적으로 활용할 수 있습니다.

### 성능
플래너를 실행하려면 먼저 `CreatePlanAsync`를 호출하고, 이후 계획을 실행해야 합니다. 이 과정에서 전체 토큰 목록을 사용하여 목표를 위한 계획을 생성하므로 시간이 소요될 수 있습니다. 또한 계획을 실행하는 데도 시간이 걸리므로, 사용자가 입력한 후 플래너의 응답을 받기 전까지 아무 반응이 없으면 앱이 응답하지 않는다고 오해할 수 있습니다. 따라서 사용자가 기다리는 동안 플래너가 피드백을 제공하거나 첫 번째 응답을 빠르게 반환하는 것이 좋습니다.

### 비용
또 다른 고려 사항은 비용입니다. 플래너의 프롬프트와 생성된 계획은 많은 토큰을 소모할 수 있습니다. 특히 복잡한 계획을 생성하는 경우 토큰 사용량이 높아져 서비스 비용이 증가할 수 있습니다. 이 문제를 줄이기 위해서는, 자주 사용하는 시나리오에 미리 정의된 계획을 사용하거나, 적은 토큰으로도 계획이 가능하도록 함수를 최적화할 필요가 있습니다.

### 정확성
플래너는 때때로 잘못된 계획을 생성할 수 있습니다. 예를 들어 변수가 잘못 전달되거나, 형식 오류가 발생하거나, 잘못된 단계를 수행할 가능성이 있습니다. 이러한 오류를 최소화하려면 오류 처리를 강화하거나, 플래너에게 계획을 '수정'하도록 요청하여 문제를 해결할 수 있습니다. 안정성을 높이기 위한 한 가지 방법은 미리 생성된 계획을 사용하는 것입니다.

### 미리 정의된 계획 사용
일반적으로 사용자들이 자주 요청하는 시나리오가 있을 수 있습니다. 이러한 경우에는 성능 저하나 비용 발생을 줄이기 위해 미리 정의된 계획을 제공하는 것이 유용합니다. 미리 정의된 계획은 자주 사용되는 시나리오에 대해 오프라인으로 생성하여 프로젝트에 저장할 수 있으며, 사용자의 요구에 따라 해당 계획을 빠르게 제공할 수 있습니다. 이를 통해 성능과 안정성을 높이고 비용을 절감할 수 있습니다.

이와 같은 사항들을 이해하고 활용하면 플래너를 통해 사용자 요구를 효율적으로 충족하는 동적인 애플리케이션을 개발할 수 있습니다.

## 자동 함수 호출

시맨틱 커널 SDK는 강력한 자동 함수 호출 함수를 지원합니다. 자동 함수 호출을 사용하면 제공하는 일부 입력에서 사용할 함수와 프롬프트를 커널이 자동으로 선택할 수 있습니다. 이 기능을 사용하면 함수와 프롬프트를 수동으로 호출할 때보다 시간을 절약하고 애플리케이션을 더 스마트하게 만들 수 있습니다.

자동 함수 호출을 사용하려면 이를 지원하는 OpenAI 모델을 사용해야 합니다. 현재 이러한 모델에는 버전 0613 이상인 GPT-3.5 터보와 GPT-4 모델이 포함됩니다.

자동 함수 호출을 사용하려면 `OpenAIPromptExecutionSettings` 개체의 `ToolCallBehavior` 속성을 `AutoInvokeKernelFunctions`에 설정해야 합니다.

예를 들어 다음 함수를 지원하는 몇 가지 성분 플러그 인이 있다고 가정해 보겠습니다.

- GetIngredients: 사용자가 보유한 재료 목록을 가져옵니다
- GetRecipe: 특정 레시피에 필요한 재료 목록을 가져옵니다
- GetMissingIngredients: 특정 레시피에 필요한 재료 중 사용자의 주방에 없는 재료 목록을 가져옵니다

```csharp
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
    "your-deployment-name",
    "your-endpoint",
    "your-api-key",
    "deployment-model");

kernel.ImportPluginFromType<IngredientsPlugin>();
kernel.ImportPluginFromPromptDirectory("Prompts/IngredientPrompts");

// Set the ToolCallBehavior property
OpenAIPromptExecutionSettings settings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

string prompt = @"What ingredients am I missing from my current list of ingredients 
    to make a recipe for aloo jeera?";

// Use the settings to automatically invoke plugins based on the prompt
var result = await kernel.InvokePromptAsync(prompt, new(settings));

Console.WriteLine(result);
```

재료 및 함수 세부 정보에 따라 이 코드의 출력은 다음 응답과 유사할 수 있습니다.

```
알루 지라 레시피에 필요한 재료 중, 당신이 가지고 있지 않은 항목은 다음과 같습니다:

쿠민 씨앗
청양고추
생강
강황 가루
고춧가루
고수 가루
신선한 고수잎
소금
기름
```

AutoInvokeKernelFunctions 설정은 프롬프트를 실행하는 데 필요한 플러그 인을 자동으로 호출하여 더 적은 코드로 애플리케이션을 동적이고 강력하게 만듭니다. 이 기능은 다양한 시나리오를 처리할 수 있는 보다 복잡한 애플리케이션을 만드는 데 도움이 될 수 있습니다.

## 자동으로 함수 호출 연습하기

시맨틱 커널 SDK를 사용하면 커널에서 참조되는 함수와 프롬프트를 자동으로 조정할 수 있습니다. 함수와 프롬프트를 수동으로 호출하는 대신 이 도구를 사용하면 시간을 절약하고 애플리케이션을 더욱 스마트하게 만들 수 있습니다. 

In [27]:
dir = Directory.GetCurrentDirectory();
kernel = builder.Build();

kernel.ImportPluginFromType<MusicLibraryPlugin>();
kernel.ImportPluginFromType<MusicConcertPlugin>();
kernel.ImportPluginFromPromptDirectory($"{dir}/Prompts");

OpenAIPromptExecutionSettings settings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

string prompt = @"저는 포틀랜드, 오리건 주에 살고 있습니다. 최근에 재생한 노래들과 다가오는 콘서트 목록을 바탕으로 어떤 콘서트를 추천해 주시겠어요?";

var result = await kernel.InvokePromptAsync(prompt, new(settings));

Console.WriteLine(result);

Based on your recent listens and current location in Portland, OR, you might enjoy attending the Kina Grannis concert. Here are the details:

- **Artist:** Kina Grannis
- **Location:** Portland, OR, USA
- **Date:** February 14, 2024

Given her indie and folk music style, it seems aligned with some of your recently played tracks. Enjoy the concert!


이전 연습에서는 함수를 수동으로 호출하거나 `Handlebars` 플래너를 사용하여 플러그 인을 함께 연결했습니다. `AutoInvokeKernelFunctions` 설정은 커널에서 참조되는 함수와 프롬프트를 자동으로 호출합니다. 이 도구를 사용하면 더 적은 코드를 사용하여 동적이고 강력한 애플리케이션을 만들 수 있습니다.